## mobilenet_v2

### import packages

In [7]:
import torch
from torch import nn
import torch.nn.functional as F

### general 'Conv' layers(head)

In [8]:
class Head(nn.Module):
    def __init__(self,inp_c,out_c):
        super(Head,self).__init__()
        self.conv2d=nn.Conv2d(inp_c,out_c,3,2,padding=1,bias=False)
        self.bn=nn.BatchNorm2d(out_c)
        self.relu6=nn.ReLU6(inplace=True)
    
    def forward(self,x):
        x=self.conv2d(x)
        x=self.bn(x)
        out=self.relu6(x)
        return out

### residual architecture(core)

In [9]:
class InvertResidual(nn.Module):
    def __init__(self,inp_c,out_c,stride,expand):
        super(InvertResidual,self).__init__()
        self.stride=stride
        out_ce=out_c*expand
        self.conv1_1=nn.Conv2d(inp_c,out_ce,1,bias=False)
        self.conv1_2=nn.Conv2d(out_ce,out_c,1,bias=False)
        self.bn1=nn.BatchNorm2d(out_ce)
        self.bn2=nn.BatchNorm2d(out_c)
        self.deepwide=nn.Conv2d(out_ce,out_ce,3,stride,padding=1,groups=out_c)
        self.relu6=nn.ReLU6(inplace=True)
        self.shortcut = nn.Sequential()
        if stride == 1 and inp_c != out_c:
            self.shortcut = nn.Sequential(
                nn.Conv2d(inp_c,out_c,1,bias=False),
                nn.BatchNorm2d(out_c))
            
        
    def forward(self,x):
        ori_x=x
        x=self.conv1_1(x)
        x=self.bn1(x)
        x=self.relu6(x)
        x=self.deepwide(x)
        x=self.bn1(x)
        x=self.relu6(x)
        x=self.conv1_2(x)
        y=self.bn2(x)
        out=y + self.shortcut(ori_x) if self.stride==1 else y
        return out

### whole net(ensemble)

In [12]:
class MobileNetv2(nn.Module):
    def __init__(self,width_mult=1,num_classes=100):
        super(MobileNetv2,self).__init__()
        block = InvertResidual
        input_channel = 32
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2]
#             [6, 64, 4, 2],
#             [6, 96, 3, 1],
#             [6, 160, 3, 2],
#             [6, 320, 1, 1]
        ]
        input_channel=int(input_channel * width_mult)
        head_layer=Head(3,input_channel)
        self.layers=[head_layer]
        for t, c, n, s in interverted_residual_setting:
            stride = s
            output_channel = int(c * width_mult)
            for i in range(n):
                if i==0:
                    self.layers.append(block(input_channel,output_channel,stride,t))
                else:
                    self.layers.append(block(input_channel,output_channel,1,t))
                input_channel=output_channel
                
        self.layers = nn.Sequential(*self.layers)
 
        self.conv_end = nn.Conv2d(32,1280, kernel_size=1, stride=1,padding=0, bias=False)
        self.bn_end = nn.BatchNorm2d(1280)
        self.relu=nn.ReLU6(inplace=True)
        self.AdaptiveAvgPool = nn.AdaptiveAvgPool2d((1, 1))
        self.linear = nn.Linear(1280, num_classes)
    def forward(self,x):
        x=self.layers(x)
        x=self.conv_end(x)
        x=self.bn_end(x)
        x=self.relu(x)
        x=self.AdaptiveAvgPool(x)
        x= x.view(x.size(0), -1)
        out = self.linear(x)
        return out

### test size

In [13]:
test_input=torch.rand(1, 3, 224, 224)
print(test_input.size())
model= MobileNetv2()
out=model(test_input )
print(out.size())

torch.Size([1, 3, 224, 224])
torch.Size([1, 100])


## VGG16

In [40]:
import torch
import torch.nn as nn
from torch.autograd import Variable
 
# cfg = {'vgg16':[64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']}
cfg = {'vgg16':[8, 8, 'M', 16, 16, 'M', 32, 32, 32, 'M', 64, 64, 64, 'M']}

In [44]:
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16,self).__init__()
        self.features = self._make_layers(cfg['vgg16'])
#         print(self.features)
        self.classifier = nn.Linear(14 * 14 * 64, 10) #主要是实现CIFAR10，不同的任务全连接的结构不同
 
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0),-1)
        x = self.classifier(x)
        return x
 
    def _make_layers(self,cfg):
        layers = []
        in_channels = 3
        for x in cfg:  #遍历list
            if x == 'M':
                layers += [nn.MaxPool2d(2,2)]
            else:
                layers += [nn.Conv2d(in_channels,x,3,1,1),
                           nn.BatchNorm2d(x),
                           nn.ReLU(inplace=True)] ##inplace为True，将会改变输入的数据 ，
                                            # 否则不会改变原输入，只会产生新的输出
                in_channels = x
        #print(layers)
        return nn.Sequential(*layers)

In [45]:
vgg16 = VGG16()

vgg16

VGG16(
  (features): Sequential(
    (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(16, 32, kernel_si

In [46]:
test_input=torch.rand(1, 3, 224, 224)
print(test_input.size())
model= VGG16()
out=model(test_input )
print(out.size())

torch.Size([1, 3, 224, 224])
torch.Size([1, 10])


In [47]:
14 * 14 * 64

12544